In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
input = '../data/data_intermediaire.parquet'

In [3]:
df = pd.read_parquet(input)
df

,text,label
0,The Sea Is Watching has been made from an orig...,1
1,There are so many reasons as to why I rate the...,1
2,The documentary presents an original theory ab...,1
3,Of all Arnold's mid-'80s movies who would have...,1
4,This Was One Scary Movie.<br /><br />Brad Pitt...,1
...,...,...
24995,This may be one of the worst movies to ever ma...,0
24996,This is certainly the worst movie i ever saw? ...,0
24997,for all the subtle charms this student film ma...,0
24998,Let's be honest shall we? Al Gore no more TRUL...,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Vectorisation du texte avec TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Entraîner un classificateur Naive Bayes
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Prédire les étiquettes sur l'ensemble de test
y_pred = model.predict(X_test_tfidf)

# Évaluer les performances du modèle
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      2485
           1       0.85      0.85      0.85      2515

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [5]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
# Préparer les données pour Word2Vec
sentences = df['text'].apply(gensim.utils.simple_preprocess).tolist()

# Entraîner un modèle Word2Vec
vector_size = 100
window = 5
min_count = 2
workers = 4

word2vec_model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

In [7]:
# Fonction pour obtenir les vecteurs de mots moyens pour chaque revue
def get_mean_word_vector(text, model):
    words = gensim.utils.simple_preprocess(text)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

In [8]:
# Appliquer la fonction pour obtenir les vecteurs de caractéristiques
df['word_vector'] = df['text'].apply(lambda x: get_mean_word_vector(x, word2vec_model))

# Convertir les vecteurs en DataFrame
vector_df = pd.DataFrame(df['word_vector'].tolist())

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(vector_df, df['label'], test_size=0.2, random_state=42)

# Entraîner un classificateur Logistic Regression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test
y_pred = classifier.predict(X_test)

# Évaluer les performances du modèle
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.81      0.80      0.81      2485
           1       0.81      0.82      0.81      2515

    accuracy                           0.81      5000
   macro avg       0.81      0.81      0.81      5000
weighted avg       0.81      0.81      0.81      5000



In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
# Fonction pour obtenir les vecteurs de mots moyens pour chaque revue
def get_mean_word_vector(text, model):
    words = gensim.utils.simple_preprocess(text)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Entraîner un modèle Word2Vec avec des paramètres par défaut
vector_size = 100
window = 5
min_count = 2
workers = 4

# Diviser les données en ensembles d'entraînement et de test
X = vector_df
y = df['label']

# Définir le pipeline
pipeline = Pipeline([
    ('classifier', LogisticRegression(max_iter=1000))
])

# Définir les paramètres à optimiser
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],  # Régularisation pour Logistic Regression
    'classifier__solver': ['liblinear', 'lbfgs']  # Solvers pour Logistic Regression
}

# Configurer GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Effectuer la recherche de grille
grid_search.fit(X, y)

# Afficher les meilleurs paramètres et le score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation croisée:", grid_search.best_score_)

# Évaluer le modèle sur l'ensemble de test avec les meilleurs paramètres
best_model = grid_search.best_estimator_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Afficher le rapport de classification
print(classification_report(y_test, y_pred))

Meilleurs paramètres: {'classifier__C': 100, 'classifier__solver': 'liblinear'}
Meilleur score de validation croisée: 0.81372
              precision    recall  f1-score   support

           0       0.81      0.80      0.81      2485
           1       0.81      0.82      0.81      2515

    accuracy                           0.81      5000
   macro avg       0.81      0.81      0.81      5000
weighted avg       0.81      0.81      0.81      5000

